In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

# 10m U

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m u/*.nc'
u_wind=xr.open_mfdataset(path)


In [3]:
u_wind

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    uwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

###### We need fix its grid size

# Resizing Grids

In [4]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
resized_uwind = u_wind.interp(lat=new_lat, lon=new_lon)

In [5]:
resized_uwind

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 73, lon: 144, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: nbnds
Data variables:
    uwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [6]:
# Comparison

# Extracting data

In [7]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_uwind = resized_uwind.sel(lat=lat_range,lon=lon_range)

In [8]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
uwind_df = pd.DataFrame(columns=['time', 'lat', 'lon', '10m_uwind'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_uwind.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_uwind.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    uwind = chunk.uwnd.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        '10m_uwind': uwind.ravel()
    })

    uwind_df = uwind_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
uwind_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,10m_uwind
0,1981-01-01,27.5,85.0,-0.288992
1,1981-01-01,25.0,87.5,-0.439931
2,1981-01-01,22.5,90.0,-0.407434
3,1981-01-01,20.0,92.5,-0.282468
4,1981-01-01,27.5,95.0,-0.567832
...,...,...,...,...
306795,2022-12-31,20.0,85.0,-0.833212
306796,2022-12-31,27.5,87.5,-0.899923
306797,2022-12-31,25.0,90.0,0.324862
306798,2022-12-31,22.5,92.5,1.591533


# 10m V

In [9]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m v/*.nc'
v_wind=xr.open_mfdataset(path)

In [10]:
v_wind

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    vwnd       (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [11]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
resized_vwind = v_wind.interp(lat=new_lat, lon=new_lon)

In [12]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_vwind = resized_vwind.sel(lat=lat_range,lon=lon_range)

In [13]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
vwind_df = pd.DataFrame(columns=['time', 'lat', 'lon', '10m_vwind'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_vwind.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_vwind.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    vwind = chunk.vwnd.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        '10m_vwind': vwind.ravel()
    })

    vwind_df = vwind_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
vwind_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,10m_vwind
0,1981-01-01,27.5,85.0,-0.803477
1,1981-01-01,25.0,87.5,0.082062
2,1981-01-01,22.5,90.0,0.217542
3,1981-01-01,20.0,92.5,-0.367649
4,1981-01-01,27.5,95.0,-0.315444
...,...,...,...,...
306795,2022-12-31,20.0,85.0,2.449651
306796,2022-12-31,27.5,87.5,3.999583
306797,2022-12-31,25.0,90.0,0.175047
306798,2022-12-31,22.5,92.5,-1.908087


In [14]:
import pandas as pd

# Assuming df_uwind and df_vwind are your dataframes
# Merge based on 'time', 'lat', and 'lon'
wind_df = pd.merge(uwind_df, vwind_df, on=['time', 'lat', 'lon'])

# If you want to merge based on an index, you can use left_index=True and right_index=True
# merged_df = pd.merge(df_uwind, df_vwind, left_index=True, right_index=True, suffixes=('_uwind', '_vwind'))


In [16]:
wind_df

,time,lat,lon,10m_uwind,10m_vwind
0,1981-01-01,27.5,85.0,-0.288992,-0.803477
1,1981-01-01,25.0,87.5,-0.439931,0.082062
2,1981-01-01,22.5,90.0,-0.407434,0.217542
3,1981-01-01,20.0,92.5,-0.282468,-0.367649
4,1981-01-01,27.5,95.0,-0.567832,-0.315444
...,...,...,...,...,...
306795,2022-12-31,20.0,85.0,-0.833212,2.449651
306796,2022-12-31,27.5,87.5,-0.899923,3.999583
306797,2022-12-31,25.0,90.0,0.324862,0.175047
306798,2022-12-31,22.5,92.5,1.591533,-1.908087


In [ ]:
sanjid

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {'A': [1, 2, 3, 4],
        'B': [5, 6, 7, 8]}

df = pd.DataFrame(data)

# Multiply a column by a constant
df['A'] = df['A'] * 2

# Multiply one column by another
# df['C'] = df['A'] * df['B']
df
# print(df)

In [ ]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m u/*.nc'
u_wind=xr.open_mfdataset(path)
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/10m v/*.nc'
v_wind=xr.open_mfdataset(path)

In [ ]:
new_lat=np.arange(90,-91,-02.5).astype(np.float32)
new_lon=np.arange(0,360,02.5).astype(np.float32)
resized_uwind=u_wind.interp(lat=new_lat,lon=new_lon)
resized_vwind=v_wind.interp(lat=new_lat,lon=new_lon)

In [ ]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_uwind = resized_uwind.sel(lat=lat_range,lon=lon_range)
bangladesh_resized_vwind = resized_vwind.sel(lat=lat_range,lon=lon_range)